In [0]:
# Check if the credential file is uploaded

dbutils.fs.ls("/FileStore/tables/")

In [0]:
file_type = "csv"
first_row_header = "true"
delimiter = ","

# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/databricks_accessKeys.csv")

In [0]:
# To send keys to AWS

from pyspark.sql.functions import *
import urllib

# Collect access and secret key from spark dataframe (dbfs:/FileStore/tables/)
ACCESS_KEY = aws_keys_df.select('Access_key_ID').collect()[0]['Access_key_ID']
SECRET_KEY = aws_keys_df.select('Secret_access_key').collect()[0]['Secret_access_key']

# Encode secret key (safe="" means every character in the secret key is encoded)
ENCODED_SECRET_KEY = urllib.parse.quote(SECRET_KEY,"")

In [0]:
# Mount the S3 bucket

# AWS S3 bucket name
AWS_S3_BUCKET = "fruits-pictures"
# Mount name for the bucket
MOUNT_NAME = "/mnt/fruits-pictures"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

In [0]:
# Read data from the mounted S3 bucket

# Check if the AWS S3 bucket was mounted successfully
# %fs ls "/mnt/fruits-pictures/"
display(dbutils.fs.ls("/mnt/fruits-pictures/test_S3/"))

In [0]:
df = spark.read.format("image").option("dropInvalid", true).load("dbfs:/mnt/fruits-pictures/test_S3/Apricot/3_100.jpg")
df.select("image.origin", "image.width", "image.height").show(truncate=False)